Imports
-------

In [11]:
from typing import List, Tuple, Union
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import sklearn.model_selection as sk
from tensorflow.keras.utils import to_categorical

Data generation
---------------

In [80]:
def generate_gaussian_data(
    num_samples, 
    input_length, 
    num_gaussian=(1, 5), 
    amplitude_range=(1, 5), 
    center_range=(32, 96), 
    width_range=(5, 20), 
    noise_amplitude=0.0):
    """
    Generate a dataset of Gaussian curves, with a variable or fixed number of peaks per slice,
    and optional Gaussian noise.

    Parameters
    ----------
    num_samples : int
        Number of slices to generate.
    input_length : int
        Length of each slice.
    num_gaussian : int or tuple of int
        If int, fixed number of Gaussian peaks per slice.
        If tuple, (min_peaks, max_peaks), a random number of peaks per slice within this range.
    amplitude_range : tuple of float
        Range of amplitudes for the Gaussian peaks.
    center_range : tuple of int
        Range of center positions for the Gaussian peaks.
    width_range : tuple of float
        Range of standard deviations (widths) for the Gaussian peaks.
    noise_amplitude : float
        Standard deviation of the Gaussian noise added to each slice.

    Returns
    -------
    X_train : numpy.ndarray
        Array of slices with Gaussian peaks and added noise, shape (num_samples, input_length, 1).
    y_train : numpy.ndarray
        Array of maximum values for each peak in every slice, shape (num_samples, max_peaks).
    num_peaks : numpy.ndarray
        Array with the number of peaks in each slice, shape (num_samples,).
    peak_positions : numpy.ndarray
        Array with the positions of peaks in each slice, shape (num_samples, max_peaks).
        Unused entries are filled with zeros.
    """
    if isinstance(num_gaussian, tuple):
        min_peaks, max_peaks = num_gaussian
    else:
        min_peaks = max_peaks = num_gaussian

    max_peaks = max(max_peaks, 1)  # Ensure at least one peak
    X_train = np.zeros((num_samples, input_length, 1))
    amplitudes = np.zeros((num_samples, max_peaks))  # Store amplitudes of peaks
    num_peaks = np.zeros(num_samples, dtype=int)  # Store the number of peaks per slice
    peak_positions = np.zeros((num_samples, max_peaks))  # Store the positions of peaks

    x = np.linspace(0, input_length - 1, input_length)

    for i in range(num_samples):
        slice_curve = np.zeros(input_length)  # Initialize the slice

        # Determine the number of peaks for this slice
        num_peaks[i] = np.random.randint(min_peaks, max_peaks + 1)
        peak_amplitudes = []
        peak_centers = []

        for _ in range(num_peaks[i]):
            amplitude = np.random.uniform(*amplitude_range)
            center = np.random.uniform(*center_range)
            width = np.random.uniform(*width_range)

            # Gaussian curve: y = A * exp(-((x - center)^2) / (2 * width^2))
            gaussian_curve = amplitude * np.exp(-((x - center) ** 2) / (2 * width ** 2))
            slice_curve += gaussian_curve  # Add the Gaussian peak to the slice
            peak_amplitudes.append(amplitude)
            peak_centers.append(center)

        # Add Gaussian noise to the slice
        noise = np.random.normal(0, noise_amplitude, input_length)
        slice_curve += noise

        # Update the arrays
        X_train[i, :, 0] = slice_curve
        amplitudes[i, :num_peaks[i]] = sorted(peak_amplitudes, reverse=True)
        peak_positions[i, :num_peaks[i]] = peak_centers

    return X_train, amplitudes, num_peaks, peak_positions


Model generation
----------------

In [94]:
def build_model(input_length, max_number_of_peaks):
    """
    Build a model that predicts both the maximum value and the position of the maximum value
    from the input signal.

    Parameters
    ----------
    input_length : int
        Length of the input signal (e.g., 128).

    Returns
    -------
    tensorflow.keras.Model
        The constructed Keras model.
    """
    input_layer = layers.Input(shape=(input_length, 1))

    # Feature extraction
    x = layers.Conv1D(filters=32, kernel_size=3, activation="relu", padding="same")(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Attention mechanism
    # attention = layers.Attention()([x, x])
    # query = layers.Dense(64)(x)  # Learnable query
    # key = layers.Dense(64)(x)    # Keys
    # value = layers.Dense(64)(x)  # Values
    attention = layers.Attention()([x, x])

    x = layers.GlobalAveragePooling1D()(attention)
    

    # Flatten and dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)

    # Output layer
    num_peaks_output = layers.Dense(max_number_of_peaks + 1, activation="softmax", name="num_peaks")(x)

    return models.Model(inputs=input_layer, outputs=num_peaks_output)

Utils
-----

In [95]:
def plot_model_performance(history, model, validation_data, num_examples=5):
    """
    Plot training history and visualize validation cases for a trained model.

    Parameters
    ----------
    history : tensorflow.keras.callbacks.History
        The training history object from model.fit().
    model : tensorflow.keras.Model
        The trained Keras model.
    validation_data : tuple
        Tuple containing validation inputs and expected outputs:
        (X_val, y_val).
    num_examples : int, optional
        Number of validation cases to visualize. Default is 5.
    """
    plt.close('all')
    
    # Unpack validation data
    X_val, y_val, peaks_positions = validation_data

    # Visualize validation cases in subplots
    num_rows = (num_examples + 1)

    fig, axes = plt.subplots(num_rows, 1, figsize=(12, num_rows * 3))

    ax = axes[0]
    twin_ax = ax.twinx()
    ax.plot(history.history['loss'], label='Training Loss', color='C0')
    twin_ax.plot(history.history['val_accuracy'], color='C1', label='Validation Accuracy')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend()
    twin_ax.legend()

    indices = np.random.choice(len(X_val), num_examples, replace=False)
    for ax, (idx, _) in zip(axes[1:], enumerate(indices)):
        input_signal = X_val[idx, :, 0]
        
        true_num_peaks = y_val[idx]
        predicted_num_peaks = model.predict(X_val[idx:idx + 1])[0]
        
        ax.plot(input_signal, label=f"GD: {true_num_peaks} -- Pred: {np.argmax(predicted_num_peaks)}")

        for peak in peaks_positions[idx]:
            if peak !=0:
                ax.axvline(peak, color='red')

        ax.legend()

    plt.tight_layout()
    plt.show()

Script
------

In [96]:
# Generate data with up to 5 peaks per slice
max_number_of_peaks = 4
gaussian_slices, amplitudes, num_peaks, positions = generate_gaussian_data(
    num_samples=400,
    input_length=128,
    num_gaussian=(1, max_number_of_peaks),
    amplitude_range=(2, 5),
    center_range=(20, 100),
    width_range=(5, 5),
    noise_amplitude=0.2
)

slices_train, slices_validation, amplitudes_train, amplitudes_validation, num_peaks_train, num_peaks_validation, positions_train, positions_validation = sk.train_test_split(
    gaussian_slices, amplitudes, num_peaks, positions, test_size=0.2, random_state=42
)


num_peaks_train_one_hot = to_categorical(num_peaks_train, max_number_of_peaks + 1)
num_peaks_validation_one_hot = to_categorical(num_peaks_validation, max_number_of_peaks + 1)


model = build_model(input_length=128, max_number_of_peaks=max_number_of_peaks)


model.compile(
    optimizer='adam',
    loss={'num_peaks': 'categorical_crossentropy',},
    metrics={'num_peaks': ['accuracy'],}
)

history = model.fit(
    slices_train,
    num_peaks_train_one_hot,
    validation_data=(slices_validation, num_peaks_validation_one_hot),
    epochs=100,
    batch_size=32
)

%matplotlib qt
plot_model_performance(history, model, (slices_validation, num_peaks_validation, positions_validation) , num_examples=5)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3005 - loss: 1.5127 - val_accuracy: 0.2750 - val_loss: 1.4980
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4857 - loss: 1.1176 - val_accuracy: 0.2750 - val_loss: 1.4345
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5428 - loss: 0.9480 - val_accuracy: 0.2750 - val_loss: 1.4041
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6050 - loss: 0.7704 - val_accuracy: 0.2750 - val_loss: 1.3833
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7959 - loss: 0.6361 - val_accuracy: 0.2750 - val_loss: 1.3775
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8286 - loss: 0.5419 - val_accuracy: 0.2750 - val_loss: 1.3925
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8294 - loss: 0.5022 - val_accuracy: 0.2750 - val_loss: 1.3882
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8293 - loss: 0.4607 - val_accuracy: 0.2750 -